In [69]:
from datetime import datetime
from pprint import pprint
import re
import sqlite3

In [2]:
db = sqlite3.connect('database.sqlite3')

In [24]:
!sqlite3 database.sqlite3 .schema

-- Loading resources from /home/remram/.sqliterc
CREATE TABLE windows(
                id INTEGER PRIMARY KEY,
                start DATETIME NOT NULL,
                end DATETIME NOT NULL,
                active BOOLEAN NOT NULL,
                name TEXT NOT NULL
            );
CREATE TABLE runs(
                id INTEGER PRIMARY KEY,
                start DATETIME NOT NULL,
                end DATETIME NULL,
                end_reason TEXT NOT NULL DEFAULT ''
            );
CREATE INDEX idx_windows_start ON windows(start);
CREATE INDEX idx_windows_end ON windows(end);
CREATE INDEX idx_windows_active ON windows(active) WHERE active=1;
CREATE INDEX idx_windows_name ON windows(name);
CREATE INDEX idx_runs_start ON runs(start);
CREATE INDEX idx_runs_end ON runs(end);


In [3]:
list(db.execute('SELECT count(id) FROM windows;'))

[(2100,)]

In [170]:
names = [name for name, in db.execute('SELECT DISTINCT name FROM windows WHERE active=1;')]
len(names)

80

In [171]:
for name in sorted(names):
    print(name)

() datamart-prod — Konsole
(NSFW) What Fetish did you have until you tried it? : AskReddit — Mozilla Firefox
- : sudo -g — Konsole
/home/remram/projects/pctracker - QGit
2017 IRONMAN Lake Placid » RTRT.me — Mozilla Firefox
ASMR Cranial Nerve Exam: Concussion Test 👩‍⚕️ (Soft Spoken Roleplay - Doctor Check Up ASMR) - YouTube — Mozilla Firefox
Add Comment | Hacker News — Mozilla Firefox
Beer distribution game - Wikipedia — Mozilla Firefox
Bevy ECS migration by cart · Pull Request #49 · bevyengine/bevy — Mozilla Firefox
Billing Statement - remirampin@gmail.com - Gmail — Mozilla Firefox
Blue Jeans Network | Video Collaboration in the Cloud - Google Chrome
CGi Artist Makes a Masterpiece Everyday For 12 Years (Today We Challenge Him) - YouTube — Mozilla Firefox
Capital One | Bank Details — Mozilla Firefox
Capture : bash — Konsole
Cerulean: Experimental micro-blogging app for Matrix | Hacker News — Mozilla Firefox
Copy of GIS_2020-12-15_Gmail - 4et50yp2ckm8otv.pdf — Mozilla Firefox
DEF CON 22 

In [108]:
def regex(pattern):
    compiled = re.compile(pattern)
    
    def regex_matcher(record):
        return compiled.search(record.name) is not None
    
    return repr(pattern), regex_matcher

In [191]:
filters = [
    (regex(' — Mozilla Firefox$'), [
        (regex(' - YouTube — Mozilla Firefox$'), []),
        (regex(' Hacker News — Mozilla Firefox$'), []),
        (regex(' GitLab — Mozilla Firefox$'), []),
        (regex(' - Gmail — Mozilla Firefox$'), []),
        (regex('Slack .* — Mozilla Firefox$'), []),
        (regex(' - Jupyter Notebook — Mozilla Firefox$'), []),
    ]),
    (regex(' - Google Chrome$'), []),
    (regex('^Signal'), []),
    (regex(' - GVIM$'), []),
    (regex(' — Konsole$'), []),
    (regex(' - QGit$'), []),
]

In [192]:
class Record(object):
    def __init__(self, start, end, name):
        self.start = datetime.fromisoformat(start)
        self.end = datetime.fromisoformat(end)
        self.name = name
        
    @property
    def duration(self):
        return (self.end - self.start).total_seconds()

In [193]:
class OutputNode(object):
    def __init__(self, name):
        self.name = name
        self.duration = 0
        self.children = {}
        
    def child(self, name):
        try:
            child = self.children[name]
        except KeyError:
            child = self.children[name] = OutputNode(name)
        return child
    
    def print(self, indent=0):
        if indent == 0:
            prefix = ''
            last_prefix = '└── '
        else:
            prefix = '│   ' * (indent - 1)
            prefix += '├── '
            last_prefix = '|   ' * indent
            last_prefix += '└── '
        name = self.name or 'total'
        other_duration = self.duration
        duration = '%ds' % int(self.duration)
        print(f'{prefix}{name} {duration}')
        children = sorted(self.children.values(), key=lambda c: -c.duration)
        for child in children:
            child.print(indent + 1)
            other_duration -= child.duration
        if self.children:
            print(f'{last_prefix}other {other_duration}')

In [194]:
def apply_filters(record, filters, output):
    # Add duration to current node
    output.duration += record.duration
    # Apply filters until one matches
    for (name, matcher), then_ops in filters:
        if matcher(record):
            apply_filters(record, then_ops, output.child(name))
            return


output = OutputNode(None)

for row in db.execute('''\
    SELECT start, end, name
    FROM windows
    WHERE active=1;
'''):
    record = Record(*row)
    apply_filters(record, filters, output)

In [195]:
output.print()

total 7737s
├── ' — Mozilla Firefox$' 5988s
│   ├── ' - YouTube — Mozilla Firefox$' 1166s
│   ├── ' - Gmail — Mozilla Firefox$' 588s
│   ├── ' - Jupyter Notebook — Mozilla Firefox$' 408s
│   ├── ' GitLab — Mozilla Firefox$' 317s
│   ├── 'Slack .* — Mozilla Firefox$' 307s
│   ├── ' Hacker News — Mozilla Firefox$' 163s
|   └── other 3039.0
├── ' — Konsole$' 664s
├── '^Signal' 223s
├── ' - GVIM$' 86s
├── ' - Google Chrome$' 26s
├── ' - QGit$' 0s
└── other 750.0
